# Nivel del Mar

Para ver el concepto de Nivel del Mar y Cambio Climático revise la [Introduccion](Introduction.ipynb).
+ Mapas de Nivel del Mar de años recientes en Magdalena.
+ Gráfico de Tendencia de Nivel del Mar en Santa Marta y Cartagena. 

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Mapas del Nive del Mar

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
    
def nivel_mar(year):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)
    img = mpimg.imread(r'./Datos/Nivel del Mar/Nivel_Mar_'+str(year)+'_ANUAL.png')
    imgplot = ax.imshow(img)
    ax.axis('off')
    #fig.suptitle('test title', fontsize=20)
    plt.title('Nivel del Mar en Noviembre de '+str(year), fontsize=15)
    plt.show()

In [ ]:
import ipywidgets as widgets
import time

a = widgets.IntSlider(min=2015,max=2018,step=1, description='Año:')
ui = widgets.HBox([a])
def f(a):
    nivel_mar(a)
    
out = widgets.interactive_output(f, {'a': a})

display(ui, out)

## Gráfico de Tendencia del Nivel del Mar 

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LinearRegression

df = pd.read_csv(r"C:\Users\verit\Documents\Nivel_del_Mar_Imagenes\Nivel_Mar_Tendencia.csv")
df.Tiempo = pd.to_datetime(df['Tiempo'], format = '%Y/%m/%d')
df = df.interpolate() #pandas interpolation for missing data

inicio = df['Tiempo'].dt.year.iloc[0]
final = df['Tiempo'].dt.year.iloc[-1]

fig = px.line(df, x="Tiempo", y=["Santa_Marta", "Cartagena"], template="seaborn")
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x unified", yaxis_title="Nivel Medio del Mar [mm]", xaxis_title="Tiempo",
                     title_text='Nivel medio del Mar en '+str(inicio)+'-'+str(final)+ ' respecto a 2013', title_x=0.22)

#Regresion Lineal
ciudades = ['Santa_Marta','Cartagena']
colores = ['#55a969','#c44e52']#'#dc8552','#4c72b0'
layoutAnnotationList = []
for ciudad,color in zip(ciudades,colores):
    regr = LinearRegression()
    x = np.array(range(len(df)))
    y = np.array(df[str(ciudad)])
    regr.fit(x.reshape(-1, 1), y.reshape(-1, 1))
    regresion = regr.predict(x.astype(float).reshape(-1, 1))
    slope_anual = regr.coef_[0][0]*12
    text=ciudad+':  y = '+str(round(slope_anual,2))+'x + '+str(round(regr.intercept_[0],2))+' [mm/año]'
    fig.add_trace((go.Scatter(x=df['Tiempo'].tolist(),y=list(regresion.ravel()), mode="lines", name='Regresion Lineal', 
                   legendgroup="group", visible = 'legendonly')))    
    layoutAnnotationList.append(dict(text=text, xref="x", yref="y",x='2016-10-01', y=min(y), showarrow=False, font=dict(color=color,size = 14)))  

    #score = regr.score(x.reshape(-1, 1), y.reshape(-1, 1))
    #print('coefficient of determination R2:', score)
    #from scipy.stats import pearsonr
    #corr, _ = pearsonr(x, y)
    #print('Pearsons correlation: %.3f' % corr)

# Agregar Botones
layoutButtons = list([dict(type="buttons",active=-1,
                           buttons=list([   
                        dict(label = 'Regresion:On',
                             method = 'update',
                             args = [{'visible': [True, True, True, True]},{'annotations':layoutAnnotationList}]),
                        dict(label = 'Regresion:Off',
                            method = 'update',
                             args = [{'visible':[True, True, False, False]},{'annotations':[]}]),
                                        ]),
                           pad={"r": 10, "t": 10},showactive=True,x=1.04,xanchor="left",
                           y=0.015 ,yanchor="bottom"
                           )
                    ])

fig.update_layout(updatemenus=layoutButtons)
fig.show()